# Housing price modeling

Elias Eskelinen, Vili Niemelä, Matti Aalto and Jarkko Komulainen

---

## Data onboarding

Fetch the data from the kaggle API (saved as a local file)

In [18]:
import os
import numpy as np
import pandas as pd
import kagglehub

# Fetch the latest version of the dataset from kaggle
data_dir = kagglehub.dataset_download("shengkunwang/housets-dataset")
data_path = os.path.join(data_dir, os.listdir(data_dir)[0]);

Load the data from the local file

In [26]:
data = pd.read_csv(data_path)
print(data.shape)

(884092, 39)


---

## Data visualization and exploratory analysis

---

## Time-series decomposition analysis

---

## Autocorrelation analysis

---

## Partitioning the time-series data